In [ ]:
# Importing necessary libraries
import mlflow
import mlflow.keras
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import shap
import numpy as np
import matplotlib.pyplot as plt
import os
import json


### Configuration and Data Loading

In this section, we load the MNIST dataset and perform normalization.


In [ ]:
# Loading the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalizing data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Expanding dimensions to add a grayscale channel
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

input_shape = x_train.shape[1:]



### Model Construction

We build a convolutional neural network model to classify MNIST images.


In [ ]:
# Building the deep learning model
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Setting up the optimizer
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


### Model Training

We define the training parameters and train the model, using MLflow to log metrics.


In [ ]:
# Training parameters
epochs = 5
batch_size = 128

# Starting an MLflow experiment
mlflow.start_run()

# Logging parameters to MLflow
mlflow.log_param("epochs", epochs)
mlflow.log_param("batch_size", batch_size)
mlflow.log_param("optimizer", "adam")
mlflow.log_param("learning_rate", 0.001)

# Trainning the model
history = model.fit(x_train, y_train,
                    validation_data=(x_test, y_test),
                    epochs=epochs,
                    batch_size=batch_size)

# Saving the model to MLflow
mlflow.keras.log_model(model, artifact_path="model")



Epoch 1/5
469/469 [==============================] - 20s 42ms/step - loss: 0.2072 - accuracy: 0.9394 - val_loss: 0.0546 - val_accuracy: 0.9830
Epoch 2/5
469/469 [==============================] - 19s 40ms/step - loss: 0.0553 - accuracy: 0.9829 - val_loss: 0.0455 - val_accuracy: 0.9856
Epoch 3/5
469/469 [==============================] - 21s 44ms/step - loss: 0.0384 - accuracy: 0.9887 - val_loss: 0.0424 - val_accuracy: 0.9867
Epoch 4/5
469/469 [==============================] - 19s 40ms/step - loss: 0.0289 - accuracy: 0.9914 - val_loss: 0.0309 - val_accuracy: 0.9898
Epoch 5/5
469/469 [==============================] - 19s 41ms/step - loss: 0.0233 - accuracy: 0.9926 - val_loss: 0.0297 - val_accuracy: 0.9899


INFO:tensorflow:Assets written to: C:\Users\JOANOB~1\AppData\Local\Temp\tmp7f7k0m34\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\JOANOB~1\AppData\Local\Temp\tmp7f7k0m34\model\data\model\assets
Setuptools is replacing distutils.


### Model Evaluation

We evaluate the model on the test set and log the final metrics to MLflow.


In [ ]:
# Evaluating the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}, Test accuracy: {test_acc}')

# Logging final metrics to MLflow
mlflow.log_metric("test_loss", test_loss)
mlflow.log_metric("test_accuracy", test_acc)


313/313 [==============================] - 2s 6ms/step - loss: 0.0297 - accuracy: 0.9899
Test loss: 0.02968001924455166, Test accuracy: 0.9898999929428101


### Model Explanation with SHAP

We generate explanations for the model's predictions using SHAP and save them in MLflow.


In [ ]:
# Creating a directory for SHAP images if it doesn't exist
shap_images_dir = "shap_images"
os.makedirs(shap_images_dir, exist_ok=True)

# Using SHAP to explain the model
background = x_train[np.random.choice(x_train.shape[0], 100, replace=False)]
test_images = x_test[:5]

# Creating the SHAP explainer and get SHAP values
explainer = shap.GradientExplainer(model, background)
shap_values = explainer.shap_values(test_images)

# Getting the model predictions
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Saving SHAP explanations in MLflow
for i in range(len(test_images)):
    shap_image = os.path.join(shap_images_dir, f"shap_explanation_{i}.png")
    shap_value = shap_values[predicted_classes[i]][i]
    
    # Visualizing and save the SHAP explanation
    shap.image_plot([shap_value], -test_images[i], show=False)
    plt.savefig(shap_image, bbox_inches='tight')
    plt.close()
    
    # Logging the image to MLflow
    mlflow.log_artifact(shap_image, artifact_path="shap_images")


`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


1/1 [==============================] - 0s 81ms/step


### End the MLflow Experiment

We end the experiment run in MLflow.


In [ ]:
# Ending the MLflow run
mlflow.end_run()